# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# import libraries
import re
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [3]:
# load data from database
engine = create_engine('sqlite:///disaster_data.db')
df = pd.read_sql_table('disaster_messages', con=engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df.shape

(26180, 40)

In [5]:
df.id.nunique()

26180

In [6]:
X = df['message'].values
Y = df.drop(['id', 'message', 'original', 'genre'], axis=1).values

In [7]:
X[:5]

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name',
       'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.',
       'says: west side of Haiti, rest of the country today and tonight'], dtype=object)

### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    # Remove punctuations
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    # tokenize text
    words = word_tokenize(text)

    # remove stopwords
    tokens = [w for w in words if w not in stopwords.words('english')]
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens

In [9]:
print(X[4])
print(tokenize(X[4]))

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [11]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f4285c90b70>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=5)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [13]:
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
# iterate thru each column to report
# f1 score, precision, recall and accuracy
for idx, col in enumerate(df.columns[4:]):
    print('*' * 60)
    print(f'Category - {col}')
    print('*' * 60)
    print(classification_report(y_test[:, idx], y_pred[:, idx]))
    print('_' * 60)
    print('Confusion Matrix')
    print(confusion_matrix(y_test[:, idx], y_pred[:, idx]))
    print('_' * 60)
    print(f'Accuracy score: {accuracy_score(y_test[:, idx], y_pred[:, idx])}')
    print('\n')

************************************************************
Category - related
************************************************************
             precision    recall  f1-score   support

          0       0.61      0.43      0.51      1225
          1       0.83      0.91      0.87      3975
          2       0.50      0.17      0.25        36

avg / total       0.78      0.80      0.78      5236

____________________________________________________________
Confusion Matrix
[[ 531  693    1]
 [ 334 3636    5]
 [   3   27    6]]
____________________________________________________________
Accuracy score: 0.7969824293353706


************************************************************
Category - request
************************************************************
             precision    recall  f1-score   support

          0       0.89      0.97      0.93      4323
          1       0.79      0.45      0.57       913

avg / total       0.87      0.88      0.87      5236

____

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[5131    2]
 [  99    4]]
____________________________________________________________
Accuracy score: 0.9807104660045837


************************************************************
Category - tools
************************************************************
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5205
          1       0.00      0.00      0.00        31

avg / total       0.99      0.99      0.99      5236

____________________________________________________________
Confusion Matrix
[[5205    0]
 [  31    0]]
____________________________________________________________
Accuracy score: 0.9940794499618029


************************************************************
Category - hospitals
************************************************************
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5184
          1       0.00      0.00      0.00        52

avg / total   

### 6. Improve your model
Use grid search to find better parameters. 

In [15]:
# Specify parameters for grid search
parameters = {
    'clf__estimator__n_estimators': [50, 100],
    'clf__estimator__min_samples_leaf': [4, 5]
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=5, verbose=4)    
cv.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] clf__estimator__min_samples_leaf=4, clf__estimator__n_estimators=50 
[CV]  clf__estimator__min_samples_leaf=4, clf__estimator__n_estimators=50, score=0.2243972308426832, total= 3.2min
[CV] clf__estimator__min_samples_leaf=4, clf__estimator__n_estimators=50 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.8min remaining:    0.0s


[CV]  clf__estimator__min_samples_leaf=4, clf__estimator__n_estimators=50, score=0.2222487467175937, total= 3.2min
[CV] clf__estimator__min_samples_leaf=4, clf__estimator__n_estimators=50 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  9.6min remaining:    0.0s


[CV]  clf__estimator__min_samples_leaf=4, clf__estimator__n_estimators=50, score=0.21795177846741465, total= 3.2min
[CV] clf__estimator__min_samples_leaf=4, clf__estimator__n_estimators=50 


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 14.3min remaining:    0.0s


[CV]  clf__estimator__min_samples_leaf=4, clf__estimator__n_estimators=50, score=0.20959656242539987, total= 3.2min
[CV] clf__estimator__min_samples_leaf=4, clf__estimator__n_estimators=50 
[CV]  clf__estimator__min_samples_leaf=4, clf__estimator__n_estimators=50, score=0.2237344794651385, total= 3.2min
[CV] clf__estimator__min_samples_leaf=4, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_leaf=4, clf__estimator__n_estimators=100, score=0.2248746717593698, total= 4.9min
[CV] clf__estimator__min_samples_leaf=4, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_leaf=4, clf__estimator__n_estimators=100, score=0.22559083313439962, total= 5.0min
[CV] clf__estimator__min_samples_leaf=4, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_leaf=4, clf__estimator__n_estimators=100, score=0.22201002625925043, total= 5.0min
[CV] clf__estimator__min_samples_leaf=4, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_leaf=4, clf

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed: 113.8min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [50, 100], 'clf__estimator__min_samples_leaf': [4, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=4)

In [19]:
print(cv.best_params_)
print(cv.best_estimator_)

{'clf__estimator__min_samples_leaf': 4, 'clf__estimator__n_estimators': 100}
Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])


In [20]:
opt_model = cv.best_estimator_
y_pred_cv = opt_model.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [21]:
# iterate thru each column to report
# f1 score, precision, recall and accuracy
for idx, col in enumerate(df.columns[4:]):
    print('*' * 60)
    print(f'Category - {col}')
    print('*' * 60)
    print(classification_report(y_test[:, idx], y_pred_cv[:, idx]))
    print('_' * 60)
    print('Confusion Matrix')
    print(confusion_matrix(y_test[:, idx], y_pred_cv[:, idx]))
    print('_' * 60)
    print(f'Accuracy score: {accuracy_score(y_test[:, idx], y_pred_cv[:, idx])}')
    print('\n')

************************************************************
Category - related
************************************************************
             precision    recall  f1-score   support

          0       0.73      0.36      0.49      1225
          1       0.82      0.96      0.89      3975
          2       1.00      0.06      0.11        36

avg / total       0.80      0.81      0.79      5236

____________________________________________________________
Confusion Matrix
[[ 447  778    0]
 [ 166 3809    0]
 [   0   34    2]]
____________________________________________________________
Accuracy score: 0.8132161955691367


************************************************************
Category - request
************************************************************
             precision    recall  f1-score   support

          0       0.87      0.99      0.93      4323
          1       0.90      0.27      0.42       913

avg / total       0.87      0.87      0.84      5236

____

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[5007    0]
 [ 229    0]]
____________________________________________________________
Accuracy score: 0.9562643239113827


************************************************************
Category - electricity
************************************************************
             precision    recall  f1-score   support

          0       0.98      1.00      0.99      5133
          1       0.00      0.00      0.00       103

avg / total       0.96      0.98      0.97      5236

____________________________________________________________
Confusion Matrix
[[5133    0]
 [ 103    0]]
____________________________________________________________
Accuracy score: 0.9803284950343774


************************************************************
Category - tools
************************************************************
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5205
          1       0.00      0.00      0.00        31

avg / total 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.